In [ ]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
from snntorch import surrogate
from snntorch import functional as SF
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import itertools
from snntorch import utils
import random

C:\Users\herna\AppData\Local\Temp\ipykernel_234196\1222478266.py:5: DeprecationWarning: The module snntorch.backprop will be deprecated in  a future release. Writing out your own training loop will lead to substantially faster performance.
  from snntorch import backprop


In [4]:
#network parameters
nin = 2
nout = 1
rand_range_start = 1
rand_range_end = 10
beta = 0.95
spike_grad = surrogate.fast_sigmoid(slope=25)

In [5]:
class EONS:
    def __init__(self,params):
        self.params = params
        self.template_network = None
        pass
    def make_template_network(self,nin:int,nout:int):
        hidden = random.randint(rand_range_start,rand_range_end)
        net = nn.Sequential(nn.Linear(nin,hidden),
                            snn.Leaky(beta=beta,spike_grad=spike_grad),
                            nn.Linear(hidden,nout),
                            nn.Leaky(beta=beta,spike_grad=spike_grad))
        self.template_network = net
        return None
    
    